# Wrangle New York OpenStreet Map
### Author: Alex Cui

In [1]:
import xml.etree.cElementTree as ET
import pprint
import re
from collections import defaultdict

## Tag names and counts

In [2]:
def count_tags(filename):
    tags = {}
    for event, element in ET.iterparse(filename):
        if element.tag not in tags.keys():
            tags[element.tag] = 1
        else:
            tags[element.tag] += 1
    return tags

In [3]:
tags = count_tags('newyork_sample.osm')
pprint.pprint(tags)

{'member': 14703,
 'nd': 1450874,
 'node': 1144670,
 'osm': 1,
 'relation': 909,
 'tag': 967033,
 'way': 179340}


## Check Tags

We first check the "k" value for each tag and see if there are any potential problems. I proposed 3 regular expressions to check for certain patterns in the "k" value.
- "lower", for tags that contain only lowercase letters and are valid
- "lower_colon", for otherwise valid tags with a colon in their names
- "problemchars", for tags with problematic characters
- "other", for other tags that do not fall into the other three categories



In [4]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

In [5]:
def key_type(element, keys):
    if element.tag == "tag":

        k = element.get("k")

        if re.search(lower,k):
            keys['lower'] += 1
        elif re.search(lower_colon,k):
            keys['lower_colon'] += 1
        elif re.search(problemchars,k):
            keys['problemchars'] += 1
        else:
            keys['other'] +=1
    
    return keys

def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)
        
    return keys

In [6]:
keys = process_map('newyork_sample.osm')
pprint.pprint(keys)

{'lower': 375253, 'lower_colon': 578910, 'other': 10370, 'problemchars': 2500}


## Find unique users
Find out how many unique users have contributed to the map in the map.

In [7]:
def get_users(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if element.get("uid"):
            users.add(element.attrib["uid"])
    return users

In [8]:
users = get_users('newyork_sample.osm')

In [9]:
len(users)

2428

## Audit street names

In [10]:
# Look for street name pattern
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

# Some expected comman street names
expected = ["Street", "Avenue", "Boulevard", 
            "Drive", "Court", "Place", 
            "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

In [11]:
# Group all the street names not expected
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

In [12]:
# Check if it is a street name
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")        

In [13]:
def audit(filename):
    
    # specify the encoding for python3
    osm_file = open(filename, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

In [14]:
street_types = audit('newyork_sample.osm')

In [15]:
# Print all the steet types and names
# We should be able to identify all problem steet names
pprint.pprint(street_types)

defaultdict(<type 'set'>, {'5': set(['Jernee Mill Rd #5']), 'Ridge': set(['Charlecote Ridge']), 'Causeway': set(['College Point Causeway']), 'Heights': set(['Columbia Heights']), 'Rd': set(['Albemarle Rd', 'Ridge Rd', 'Commack Rd']), 'Turnpike': set(['Union Turnpike', 'Hempstead Turnpike', 'Jericho Turnpike', 'Heampstead Turnpike', 'North Franklin Turnpike']), 'Concourse': set(['Grand Concourse']), 'STREET': set(['BARTLETT STREET']), 'Rb': set(['Linden Boulevard Outer Eb Rb']), 'Gate': set(['West Gate', 'Ridings Gate', 'Martingale Gate', 'Rutland Gate']), 'avenue': set(['Bedford avenue']), 'M204': set(['3rd Ave #M204']), 'Southwest': set(['Prospect Park Southwest']), 'Promenade': set(['Mayberry Promenade']), 'Rdg': set(['Ring Neck Rdg']), 'Mews': set(['Washington Mews']), '22': set(['US- 22', 'US 22', 'Route 22']), 'Cove': set(['Yacht Club Cove']), 'Concrs': set(['Western Concrs']), 'Crst': set(['Dianne Crst']), 'Path': set(['Candlewood Path', 'Lone Oak Path', 'Straight Path', 'Pilgrim

In [16]:
# Fix these street names (wrong name: correct name)
mapping = { "AVENUE": "Avenue",
            "AVenue": "Avenue",
            "Ave": "Avenue",
            "Ave.": "Avenue",
            "Avene": "Avenue",
            "Blvd": "Boulevard",
            "Ct": "Court",
            "DRIVE": "Drive",
            "Dr": "Drive",
            "Pkwy": "Parkway",
            "Plz": "Plaza",
            "ROAD": "Road",
            "Rd": "Road",
            "STREET": "Street",            
            "St": "Street",
            "St.": "Street",
            "Steet": "Street",
            "Trce": "Terrace",
            "Tpke": "Turnpike",
            "avenue": "Avenue",
            "street": "Street"
            }


In [17]:
def update_streetname(name, mapping):
    m = street_type_re.search(name)
    better_name = name
    # condition: if the street name does have a last word
    if m:
        # check if the street type is a key in your mapping dictionary:
        if m.group() in mapping.keys():
            better_street_type = mapping[m.group()]
            better_name = street_type_re.sub(better_street_type, name)
    return better_name

In [18]:
# Fix the street names
for st_type, ways in street_types.items():
        for name in ways:
            better_name = update_streetname(name, mapping)
            print (name, "=>", better_name)

('Jernee Mill Rd #5', '=>', 'Jernee Mill Rd #5')
('Charlecote Ridge', '=>', 'Charlecote Ridge')
('College Point Causeway', '=>', 'College Point Causeway')
('Columbia Heights', '=>', 'Columbia Heights')
('Albemarle Rd', '=>', 'Albemarle Road')
('Ridge Rd', '=>', 'Ridge Road')
('Commack Rd', '=>', 'Commack Road')
('Union Turnpike', '=>', 'Union Turnpike')
('Hempstead Turnpike', '=>', 'Hempstead Turnpike')
('Jericho Turnpike', '=>', 'Jericho Turnpike')
('Heampstead Turnpike', '=>', 'Heampstead Turnpike')
('North Franklin Turnpike', '=>', 'North Franklin Turnpike')
('Grand Concourse', '=>', 'Grand Concourse')
('BARTLETT STREET', '=>', 'BARTLETT Street')
('Linden Boulevard Outer Eb Rb', '=>', 'Linden Boulevard Outer Eb Rb')
('West Gate', '=>', 'West Gate')
('Ridings Gate', '=>', 'Ridings Gate')
('Martingale Gate', '=>', 'Martingale Gate')
('Rutland Gate', '=>', 'Rutland Gate')
('Bedford avenue', '=>', 'Bedford Avenue')
('3rd Ave #M204', '=>', '3rd Ave #M204')
('Prospect Park Southwest', '=>

### Check the postal code
New York area post codes are 5-digit which range from 10001-11102. Let's first print the post codes which are not in this format.

In [19]:
ny_postcode_range = ["00501","14925"]

In [20]:
def audit_postcode(invalid_postcode_types, postcode):
    try: 
        if not (int(ny_postcode_range[0]) <= int(postcode) <= int(ny_postcode_range[1])):
                raise ValueError
    except ValueError:
        invalid_postcode_types[postcode] += 1
        
def is_postcode(elem):
    return (elem.attrib['k'] == "addr:postcode")

In [21]:
def audit(filename):
    
    # specify the encoding for python3
    osm_file = open(filename, "r")
    invalid_postcode_types = defaultdict(int)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_postcode(tag):
                    audit_postcode(invalid_postcode_types, tag.attrib['v'])
    osm_file.close()
    return invalid_postcode_types

In [22]:
invalid_postcode_types = audit("newyork_sample.osm")

In [23]:
pprint.pprint(invalid_postcode_types)

defaultdict(<type 'int'>, {'08854-8037': 2, '08901-8554': 1, '08854-8035': 1, '08901-1004': 1, '08901-2890': 1, '08901-8553': 1, '08901-1340': 1, '08901-8539': 1, '08901-1175': 1, '08854-8039': 1, '08854-8038': 1, '08854-8051': 1, '08854-8053': 1, '08854-8052': 1, '08854-8058': 2, '22645': 1, '90745': 1, '08854-5627': 1, '08901-8541': 1, '08901-8502': 2, '100014': 1, '10002-1013': 1, '10017-6927': 1, '115422': 1, '08901-8528': 1, '08854-8009': 1, '08901-8524': 1, '08854-8041': 1, '08901-1164': 1, '08901-8520': 2, '08854-8000': 1, '08854-8064': 2, '08901-1108': 2, '08854-8067': 1, '08901-8504': 1, '08854-8062': 1, '08854-8063': 1, '08854-8012': 1, 'NY 10533': 24, 'CT 06870': 1, 'NY 11201': 1, '10314-3903': 1, '08901-2867': 1})


In [24]:
ny_postcode_default = "10001"

def update_postcode(postcode):
    try:
        if not (int(ny_postcode_range[0]) <= int(postcode) <= int(ny_postcode_range[1])):
            raise ValueError
        else:
            return str(postcode)
    except ValueError:
        return ny_postcode_default

In [25]:
for postcode in invalid_postcode_types.keys():
    better_postcode = update_postcode(postcode)
    print (postcode, "=>", better_postcode)

('08854-8037', '=>', '10001')
('08901-8554', '=>', '10001')
('08854-8035', '=>', '10001')
('08901-1004', '=>', '10001')
('08901-2890', '=>', '10001')
('08901-8553', '=>', '10001')
('08901-1340', '=>', '10001')
('08901-8539', '=>', '10001')
('08901-1175', '=>', '10001')
('08854-8039', '=>', '10001')
('08854-8038', '=>', '10001')
('08854-8051', '=>', '10001')
('08854-8053', '=>', '10001')
('08854-8052', '=>', '10001')
('08854-8058', '=>', '10001')
('22645', '=>', '10001')
('90745', '=>', '10001')
('08854-5627', '=>', '10001')
('08901-8541', '=>', '10001')
('08901-8502', '=>', '10001')
('100014', '=>', '10001')
('10002-1013', '=>', '10001')
('10017-6927', '=>', '10001')
('115422', '=>', '10001')
('08901-8528', '=>', '10001')
('08854-8009', '=>', '10001')
('08901-8524', '=>', '10001')
('08854-8041', '=>', '10001')
('08901-1164', '=>', '10001')
('08901-8520', '=>', '10001')
('08854-8000', '=>', '10001')
('08854-8064', '=>', '10001')
('08901-1108', '=>', '10001')
('08854-8067', '=>', '10001'

## Generate the csv

In [26]:
import csv
import codecs
import cerberus
import schema

In [35]:
# File path

OSM_PATH = "newyork_sample.osm"

NODES_PATH = "nodes_emptylines.csv"
NODE_TAGS_PATH = "nodes_tags_emptylines.csv"
WAYS_PATH = "ways_emptylines.csv"
WAY_NODES_PATH = "ways_nodes_emptylines.csv"
WAY_TAGS_PATH = "ways_tags_emptylines.csv"

In [36]:
LOWER_COLON = re.compile(r'^([a-zA-Z0-9]|_)+:([a-zA-Z0-9]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
SCHEMA = schema.schema

In [37]:
# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

In [38]:
# ================================================== #
#               Helper Functions                     #
# ================================================== #

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    # YOUR CODE HERE
    if element.tag == 'node':
        for i in node_attr_fields:
            node_attribs[i] = element.attrib[i]
            
    if element.tag == 'way':
        for i in way_attr_fields:
            way_attribs[i] =element.attrib[i]
            
    for tag in element.iter('tag'):
        dic = {}
        
        if problem_chars.search(tag.attrib['k']):
            continue
        
        if element.tag == 'node':
            dic['id'] = node_attribs['id']
        elif element.tag == 'way':
            dic['id'] = way_attribs['id']
            
        ##dic['value'] = tag.attrib['v']
        
        if LOWER_COLON.search(tag.attrib['k']):
            dic['type'] = re.search('[a-zA-Z0-9]*:', tag.attrib['k']).group()[:-1] 
            dic['key'] = re.search(':([a-zA-Z0-9]|_)+:?([a-zA-Z0-9]|_)*$', tag.attrib['k']).group()[1:]
            
            if dic['key'] == 'street' and dic['type'] == 'addr':
                dic['value'] = update_streetname(tag.attrib['v'],mapping)
                
            elif dic['key'] == 'postcode' and dic['type'] == 'addr':
                dic['value'] = update_postcode(tag.attrib['v'])
                
            else:
                dic['value'] = tag.attrib['v']
                           
        else:
            dic['key'] = tag.attrib['k']
            dic['type'] = 'regular'
            dic['value'] = tag.attrib['v']

        tags.append(dic)

    if element.tag == 'way':
        count = 0 
    
        for nd in element.iter('nd'):
            way_node_dict = {}
            way_node_dict['id'] = way_attribs['id']
            way_node_dict['node_id'] = nd.attrib['ref']
            way_node_dict['position'] = count
            count += 1
        
            way_nodes.append(way_node_dict)     
        
    if element.tag == 'node':   
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()
            
def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))
        
class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.items()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

In [39]:
# ================================================== #
#               Main Function                        #
# ================================================== #

# -*- coding: utf-8 -*-

def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        ##nodes_writer.writeheader()
        ##node_tags_writer.writeheader()
        ##ways_writer.writeheader()
        ##way_nodes_writer.writeheader()
        ##way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])

In [40]:
process_map(OSM_PATH, validate=True)

In [41]:
import glob

In [42]:
# Clean the empty lines

def clear_empty(filenames):
    for filename in filenames:
        input = open(filename, 'rb')
        output = open(filename[:-15]+'.csv', 'wb')
        writer = csv.writer(output)
        for row in csv.reader(input):
            if any(row):
                writer.writerow(row)
        input.close()
        output.close()

In [43]:
filenames = glob.glob("*.csv")
clear_empty(filenames)

# SQL

In [2]:
import sqlite3
import pprint

In [3]:
# Fetch records from newyork.db
db = sqlite3.connect("newyork.db")
c = db.cursor()

In [6]:
# Number of nodes
QUERY = "SELECT COUNT(*) FROM nodes;"
c.execute(QUERY)
rows = c.fetchall()
print rows

[(1048575,)]


In [7]:
# Number of ways
QUERY = "SELECT COUNT(*) FROM ways;"
c.execute(QUERY)
rows = c.fetchall()
print rows

[(179340,)]


In [9]:
# Number of unique users(id)
QUERY = " SELECT COUNT(*) \
          FROM (SELECT uid from nodes UNION SELECT uid FROM ways);"

c.execute(QUERY)
rows = c.fetchall()
pprint.pprint (rows)

[(2394,)]
